使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第一節 全般）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np

import s03_01_hc_load_general as ld
from s03_01_hc_load_general import calc_etr_dash_t

from s04_01_general import get_heating_mode, get_default_heating_spec, get_default_heatsource

import s09_03_air_based_solar_system as ass

# ダクト式セントラル空調機
import s04_02_central_heating_system_with_duct as dc
import s04_02_central_heating_system_with_duct as dc_spec

# エアーコンディショナー
import s04_03_room_air_conditioner as rac
import s04_03_room_air_conditioner as rac_spec

# FF暖房
import s04_04_ff_type_heating as ff
import s04_04_ff_type_heating as ff_spec

# 電気ヒーター床暖房
import s04_05_electric_floor_heating as eheater
import s04_05_electric_floor_heating as eheater_spec

# 電気蓄熱暖房
import s04_06_electric_thermal_storage_type_heating as ets
import s04_06_electric_thermal_storage_type_heating as ets_spec

# 温水暖房
import s04_07_a_kerosene_boiler as hwh

# 温水暖房用パネルラジエーター
import s04_07_j_panel_radiator as rad_panel

# 温水暖房用ファンコンベクター
import s04_07_k_fan_convector as rad_fanc

# 温水暖房用床暖房
import s04_07_l_hot_water_floor_heating as rad_floor

# ルームエアコンディショナー付温水床暖房
import s04_08_hot_water_floor_heating_with_rac as racfh
import s04_08_hot_water_floor_heating_with_rac as racfh_spec

# 第四章　暖冷房設備 / Chapter 4  Heating and cooling equipment

## 第一節　全般 / Section 1 General

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．暖房方式及び冷房方式 / Heating method and cooling method

省略

Omission

### 6．暖房設備の一次エネルギー消費量及び処理負荷と未処理負荷 / Primary energy consumption of heating equipment and processing load and unprocessed load

In [ ]:
# 暖房負荷の取得
def get_heating_load(region, sol_region, A_A, A_MR, A_OR, Q, mu_H, TS, r_A_ufvnt, HEX, underfloor_insulation, mode_H,
                     spec_MR, spec_OR, mode_MR, mode_OR, SHC):
    if mode_H == '住戸全体を連続的に暖房する方式':
        # 住宅全体を連続的に暖房する方式おける暖房負荷 (2)
        L_H_A_d_t = get_L_H_A_d_t(spec_MR, spec_OR, SHC, A_A, A_MR, A_OR, region, sol_region, Q, mu_H, TS,
                                  r_A_ufvnt,
                                  HEX, underfloor_insulation)
        return L_H_A_d_t, None, None
    elif mode_H == '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合':
        # 暖房区画i=1-5それぞれの暖房負荷
        L_T_H_d_t_i, L_dash_H_R_d_t_i = get_L_H_d_t(region, sol_region, A_A, A_MR, A_OR, None, spec_MR, spec_OR,
                                                    mode_MR, mode_OR, Q,
                                                    mu_H, TS, r_A_ufvnt, HEX, SHC, underfloor_insulation)
        return None, L_T_H_d_t_i, L_dash_H_R_d_t_i
    elif mode_H is None:
        return None, None, None
    else:
        raise ValueError(mode_H)

#### 6.1 処理負荷及び未処理負荷 / Processing load and unprocessed load

#### 6.1.1 住戸全体を連続的に暖房する方式 / Method to heat the whole dwelling unit continuously

$$
\large
Q_{UT,H,A,d,t} =\sum_{i=1}^{5} Q_{UT,H,d,t,i}
$$

$Q_{UT,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷(MJ/h) / Raw heating load of heating equipment per hour at time $ t $ of date $ d $ in the scheme of heating the whole dwelling unit continuously  
$Q_{UT,H,A,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器等の未処理暖房負荷(MJ/h) /
Unprocessed heating load of heating equipment installed in heating / cooling section $ i $ per hour at time $ t $ on date $ d $

In [ ]:
def calc_Q_UT_H_A_d_t(A_A, A_MR, A_OR, A_env, mu_H, mu_C, q_hs_rtd_H, q_hs_rtd_C, V_hs_dsgn_H, V_hs_dsgn_C, Q,
                     VAV, general_ventilation, duct_insulation, region, L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i):
    """住宅全体を連続的に暖房する方式おける暖房設備の未処理暖房負荷 (1)

    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param A_env: 外皮の部位の面積の合計 (m2)
    :type A_env: float
    :param mu_H: 当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_H: float
    :param mu_C: 当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_C: float
    :param q_hs_rtd_H: 熱源機の暖房時の定格出力 (MJ/h)
    :type q_hs_rtd_H: float
    :param q_hs_rtd_C: 熱源機の冷房時の定格出力 (MJ/h)
    :type q_hs_rtd_C: float
    :param V_hs_dsgn_H: 暖房時の設計風量（m3/h）
    :type V_hs_dsgn_H: float
    :param V_hs_dsgn_C: 冷房時の設計風量（m3/h）
    :type V_hs_dsgn_C: float
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :type Q: float
    :param VAV: VAV有無
    :type VAV: bool
    :param general_ventilation: 全版換気の機能の有無
    :type general_ventilation: bool
    :param duct_insulation: ダクトが通過する空間
    :type duct_insulation: str
    :param region: 省エネルギー地域区分
    :type region: int
    :param L_H_d_t_i: 日付dの時刻tにおける暖冷房区画iの1時間当たりの暖房負荷（MJ/h）
    :type L_H_d_t_i: ndarray
    :param L_CS_d_t_i: 暖冷房区画iの 1 時間当たりの冷房顕熱負荷
    :type L_CS_d_t_i: ndarray
    :param L_CL_d_t_i: 暖冷房区画iの 1 時間当たりの冷房潜熱負荷
    :type L_CL_d_t_i: ndarray
    :return: 住戸全体を連続的に暖房する方式における1時間当たりの暖房設備の未処理暖房負荷(MJ/h)
    :rtype: ndarray
    """
    _, Q_UT_H_d_t_i, _, _, _, _, _, _, _, _, _ = dc.calc_Q_UT_A(A_A, A_MR, A_OR, A_env, mu_H, mu_C, q_hs_rtd_H, q_hs_rtd_C,
                                          V_hs_dsgn_H, V_hs_dsgn_C, Q, VAV, general_ventilation,
                                          duct_insulation, region, L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i)

    Q_UT_H_A_d_t = np.sum(Q_UT_H_d_t_i, axis=0)

    return Q_UT_H_A_d_t

#### 6.1.2 居室のみを暖房する方式 / Method to heat only room

$$
\large
Q_{T,H,MR,d,t} = Q_{T,H,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,H,MR,d,t} = Q_{UT,H,d,t,i}|_{i=1}
$$

$Q_{T,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の処理暖房負荷(MJ/h) / Processing heating load of the heating facility installed in the main room per hour at the time $ t $ of date $ d $ in the method of heating only the room  
$Q_{UT,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の未処理暖房負荷(MJ/h) / Raw heating load of the heating equipment installed in the main room per hour at the time $ t $ of date $ d $ in the method of heating only the room  
$Q_{T,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器等の処理暖房負荷(MJ/h) / Processing heating load such as heating equipment installed in heating / cooling section $i$ per hour at time $ t $ of date $ d $  
$Q_{UT,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器等の未処理暖房負荷(MJ/h) / Raw heating load of heating equipment installed in heating / cooling section $i$ per hour at time $ t $ of date $ d $  

In [ ]:
#主たる居室に設置された暖房設備の処理暖房負荷
def get_Q_T_H_MR_d_t():
     return get_Q_T_H_d_t_i(i=1)

# 主たる居室に設置された暖房設備の未処理暖房負荷
def get_Q_UT_H_MR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:

        # 送水温度の決定
        Theta_SW_hs_op = hwh.get_Theta_SW_hs_op(spec_HS['type'], CG)
        rad_list = hwh.get_rad_list(spec_MR, spec_OR)
        p_hs_d_t = hwh.get_p_hs_d_t(Theta_SW_hs_op, rad_list, L_T_H_d_t, A_A, A_MR, A_OR, region, mode_MR, mode_OR)
        Theta_SW_d_t = hwh.get_Theta_SW_d_t(Theta_SW_hs_op, p_hs_d_t)

        if spec_MR['type'] == '温水暖房用パネルラジエーター':
            # 床面積
            A_HCZ = get_A_HCZ_i(1, A_A, A_MR, A_OR)
            R_type = '主たる居室'

            # パネルラジエーターの最大能力
            q_max_rad = rad_panel.get_q_max_rad(region, mode_MR, A_HCZ, R_type)
            # パネルラジエーターの最大暖房出力
            Q_max_H_rad = rad_panel.get_Q_max_H_rad(Theta_SW_d_t, q_max_rad)
        elif spec_MR['type'] == '温水暖房用床暖房':
            # 床面積
            A_HCZ = get_A_HCZ_i(1, A_A, A_MR, A_OR)
            r_Af = spec_MR['r_Af']
            A_f = rad_floor.get_A_f(A_HCZ, r_Af)
            # 温水床暖房の単位面積当たりの上面最大放熱能力
            Q_max_H_rad = rad_floor.get_Q_max_H_rad(Theta_SW_d_t, A_f)
        elif spec_MR['type'] == '温水暖房用ファンコンベクター':
            # 床面積
            A_HCZ = get_A_HCZ_i(1, A_A, A_MR, A_OR)
            R_type = '主たる居室'
            q_max_FC = rad_fanc.get_q_max_FC(region, mode_MR, A_HCZ, R_type)
            Q_max_H_rad = rad_fanc.get_Q_max_H_rad(Theta_SW_d_t, q_max_FC)
        else:
            raise ValueError(spec_MR['type'])

        # 処理負荷
        Q_T_H_d_t_i = np.min([Q_max_H_rad, L_T_H_d_t[0]], axis=0)

        # 未処理負荷
        Q_UT_H_d_t_i = L_T_H_d_t[0] - Q_T_H_d_t_i

        return Q_UT_H_d_t_i
    else:
        return calc_Q_UT_H_d_t(1, spec_MR, A_A, A_MR, A_OR, region, mode_MR, L_T_H_d_t[0])

$$
\large
Q_{T,H,OR,d,t} = \sum_{i=2～5} Q_{T,H,d,t,i} 
$$

$$
\large
Q_{UT,H,OR,d,t} = \sum_{i=2～5} Q_{UT,H,d,t,i} 
$$

$Q_{T,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の処理暖房負荷(MJ/h) / Processing heating load of heating facilities installed in other rooms per hour at the time $ t $ of date $ d $ in the method of heating only the room  
$Q_{UT,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の未処理暖房負荷(MJ/h) / Raw heating load of heating facilities installed in other rooms per hour at the time $ t $ of date $ d $ in the method of heating only the room  

In [ ]:
# その他の居室に設置された暖房設備の処理暖房負荷
def get_Q_T_H_OR_d_t():
    return np.sum([get_Q_T_H_d_t_i(i) for i in range(2, 6)], axis=0)

# その他の居室に設置された暖房設備の未処理暖房負荷
def get_Q_UT_H_OR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t):
    # その他の居室がない場合
    if A_OR == 0:
        return np.zeros(24 * 365)
    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            # 送水温度の決定
            Theta_SW_hs_op = hwh.get_Theta_SW_hs_op(spec_HS['type'], CG)
            rad_list = hwh.get_rad_list(spec_MR, spec_OR)
            p_hs_d_t = hwh.get_p_hs_d_t(Theta_SW_hs_op, rad_list, L_T_H_d_t, A_A, A_MR, A_OR, region, mode_MR,
                                        mode_OR)
            Theta_SW_d_t = hwh.get_Theta_SW_d_t(Theta_SW_hs_op, p_hs_d_t)

            # 未処理負荷
            Q_UT_H_d_t_i = np.zeros((5, 24 * 365))
            for i in range(2, 6):

                if spec_OR['type'] == '温水暖房用パネルラジエーター':
                    # 床面積
                    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
                    R_type = 'その他の居室'

                    # パネルラジエーターの最大能力
                    q_max_rad = rad_panel.get_q_max_rad(region, mode_OR, A_HCZ, R_type)
                    # パネルラジエーターの最大暖房出力
                    Q_max_H_rad = rad_panel.get_Q_max_H_rad(Theta_SW_d_t, q_max_rad)
                elif spec_OR['type'] == '温水暖房用床暖房':
                    # 床面積
                    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
                    r_Af = spec_OR['r_Af']
                    A_f = rad_floor.get_A_f(A_HCZ, r_Af)
                    # 温水床暖房の単位面積当たりの上面最大放熱能力
                    Q_max_H_rad = rad_floor.get_Q_max_H_rad(Theta_SW_d_t, A_f)
                elif spec_OR['type'] == '温水暖房用ファンコンベクター':
                    # 床面積
                    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
                    R_type = 'その他の居室'
                    q_max_FC = rad_fanc.get_q_max_FC(region, mode_OR, A_HCZ, R_type)
                    Q_max_H_rad = rad_fanc.get_Q_max_H_rad(Theta_SW_d_t, q_max_FC)
                else:
                    raise ValueError(spec_OR['type'])

                # 処理負荷
                Q_T_H_d_t_i = np.min([Q_max_H_rad, L_T_H_d_t[i - 1]], axis=0)

                # 未処理負荷
                Q_UT_H_d_t_i[i - 1] = L_T_H_d_t[i - 1] - Q_T_H_d_t_i

            return np.sum(Q_UT_H_d_t_i, axis=0)
        else:
            return np.sum(
                [calc_Q_UT_H_d_t(i, spec_OR, A_A, A_MR, A_OR, region, mode_OR, L_T_H_d_t[i - 1]) for i in
                 range(2, 6)], axis=0)


def calc_Q_UT_H_d_t(i, device, A_A, A_MR, A_OR, region, mode, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':

        # 仕様の取得
        q_rtd_C = rac_spec.get_q_rtd_C(A_HCZ)
        q_rtd_H = rac_spec.get_q_rtd_H(q_rtd_C)
        e_rtd_C = rac_spec.get_e_rtd_C(device['e_class'], q_rtd_C)
        e_rtd_H = rac_spec.get_e_rtd_H(e_rtd_C)

        # 未処理負荷の計算
        Q_UT_H_d_t = rac.get_Q_UT_H_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t,
        )

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        q_min_H = ff_spec.get_q_min_H(q_max_H)
        P_rtd_H = ff_spec.get_P_rtd_H(q_max_H)
        P_itm_H = ff_spec.get_P_itm_H()

        # 未処理負荷の計算
        Q_UT_H_d_t = ff.get_Q_UT_H_d_t(
            q_max_H=q_max_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':

        # 仕様の取得
        q_rq_H = ets_spec.get_q_rq_H(region)
        f_cT = ets_spec.get_f_cT(region)
        f_cI = ets_spec.get_f_cI(mode, '主たる居室' if i == 1 else 'その他の居室')
        q_rtd_H = ets_spec.get_q_rtd_H(q_rq_H, A_HCZ, f_cT, f_cI)
        e_rtd_H = ets_spec.get_e_rtd_H()

        # 未処理負荷の計算
        Q_UT_H_d_t = ets.get_Q_UT_H_d_t(
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気ヒーター床暖房':

        # 仕様の取得
        r_Af = device['r_Af']
        r_up = device['r_up']
        A_f = eheater_spec.get_A_f(A_HCZ, r_Af)

        # 未処理負荷の計算
        Q_UT_H_d_t = eheater.get_Q_UT_H_d_t(
            A_f=A_f,
            r_up=r_up,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        raise ValueError(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':

        # 仕様の取得
        r_Af = device['r_Af']

        # 未処理負荷の計算
        Q_UT_H_d_t = racfh.get_Q_UT_H_d_t(region, A_HCZ, r_Af, L_H_d_t)

    else:
        raise ValueError(device['type'])

    return Q_UT_H_d_t

$$
\large
Q_{T,H,d,t,i} = min(Q_{max,H,d,t,i}, L_{H,d,t,i})
$$

$$
\large
Q_{UT,H,d,t,i} = L_{H,d,t,i} - Q_{T,H,d,t,i}
$$

$Q_{max,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器等の最大暖房出力(MJ/h) / Maximum heating output of heating equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$L_{H,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの暖房負荷(MJ/h) / Heating load per hour of heating / cooling division $ i $ at time $ t $ of date $ d $  

$L_{H,d,t,i}$が0未満の場合は0とする。 

If $ L_ {H, d, t, i} $ is less than 0, it is 0.

In [ ]:
# 暖冷房区画iに設置された暖房設備機器等の処理暖房負荷 
def get_Q_T_H_d_t_i(Q_max_H_d_t_i, L_H_d_t_i):
    assert len(Q_max_H_d_t_i.shape) == 1
    assert len(L_H_d_t_i.shape) == 1
    return np.min([Q_max_H_d_t_i, L_H_d_t_i], axis=0)

# 暖冷房区画iに設置された暖房設備機器等の未処理暖房負荷
def get_Q_UT_H_d_t_i(Q_T_H_d_t_i, L_H_d_t_i):
    # 1 時間当たりの暖房負荷
    __L_H_d_t_i = np.max([np.zeros(24 * 365), L_H_d_t_i], axis=0)  # 0未満の場合は0

    return __L_H_d_t_i - Q_T_H_d_t_i

#### 6.2 暖房設備のエネルギー消費量 / Energy consumption of heating equipment

$$
\large
E_{E,H,d,t} = E_{E,H,hs,d,t} + E_{E,H,aux,ass,d,t}
$$

$$
\large
E_{G,H,d,t} = E_{G,H,hs,d,t}
$$

$$
\large
E_{K,H,d,t} = E_{K,H,hs,d,t}
$$

$$
\large
E_{M,H,d,t} = E_{M,H,hs,d,t}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ of date $ d $  
$E_{G,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{K,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の灯油消費量(MJ/h) / Kerosene consumption of heating equipment per hour at time $ t $ of date $ d $  
$E_{M,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of heating installation per hour at time $ t $ of date $ d $  
$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等の消費電力量(kWh/h) / Power consumption of heating equipment etc. per hour at time $ t $ of date $ d $  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等の灯油消費量(MJ/h) / Consumption of kerosene such as heating equipment per hour at time $ t $ on date $ d $  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels such as heating equipment per hour at time $ t $ on date $ d $  
$E_{E,H,aux,ass,d,t}$：日付$d$の時刻$t$における1時間当たりの空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分(kWh/h) / Addition to the heating equipment of the energy consumption of the auxiliary equipment of the air collection solar utilization equipment per hour at the time $ t $ of the date $ d $  

日付$d$の時刻$t$における1 時間当たりの空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分$E_{E,H,aux,ass,d,t}$は、空気集熱式太陽熱利用設備を採用しない場合はゼロとする。

Of the energy consumption of the air-collecting solar thermal energy equipment per hour at the time $ t $ of the date $ d $, additional to the heating equipment $ E_ {E, H, aux, ass, d, t} $ is assumed to be zero if the air collecting solar system is not adopted.

In [ ]:
# 暖房設備の消費電力量（kWh/h）
def get_E_E_H_d_t(region, sol_region, A_A, A_MR, A_OR,
                  H_A=None,
                  spec_MR=None,
                  spec_OR=None, spec_HS=None,
                  mode_MR=None, mode_OR=None,
                  CG=None,
                  SHC=None, heating_flag_d=None,
                  L_H_A_d_t=None, L_T_H_d_t_i=None,
                  **args):

    if region == 8:
        return np.zeros(24 * 365)

    # 暖房設備機器等の消費電力量
    E_E_hs_d_t = get_E_E_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG,
                                L_H_A_d_t, L_T_H_d_t_i)

    # 空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分
    E_E_aux_ass_d_t = get_E_E_aux_ass_d_t(SHC, heating_flag_d, region, sol_region)

    return E_E_hs_d_t + E_E_aux_ass_d_t


# 空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分
def get_E_E_aux_ass_d_t(SHC, heating_flag_d, region, sol_region):
    if SHC is not None and SHC['type'] == '空気集熱式':
        E_E_aux_ass_d_t = ass.calc_E_E_H_aux_ass_d_t(
            hotwater_use=SHC['hotwater_use'],
            heating_flag_d=heating_flag_d,
            region=region,
            sol_region=sol_region,
            P_alpha=SHC['P_alpha'],
            P_beta=SHC['P_beta'],
            A_col=SHC['A_col'],
            V_fan_P0=SHC['V_fan_P0'],
            m_fan_test=SHC['m_fan_test'],
            d0=SHC['d0'],
            d1=SHC['d1'],
            fan_sso=SHC['fan_sso'],
            fan_type=SHC['fan_type'],
        )
    else:
        E_E_aux_ass_d_t = np.zeros(24 * 365)

    return E_E_aux_ass_d_t


# 暖房設備機器等の消費電力量（kWh/h）
def get_E_E_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_H_A_d_t,
                   L_T_H_d_t_i):
    if H_A is not None:
        return get_E_E_H_hs_A_d_t(region, A_A, H_A, L_H_A_d_t)
    elif (spec_MR is not None or spec_OR is not None) and L_T_H_d_t_i is not None:
        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合 (8a)
            return get_E_E_H_hs_MROR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, CG, L_T_H_d_t_i)
        else:
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合 (9a)
            return get_E_E_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                       L_T_H_d_t_i, CG) \
                   + get_E_E_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                         L_T_H_d_t_i, CG)
    else:
        return np.zeros(24*365)


def is_hotwaterheatingonly(H_MR, H_OR):
    from section4_7 import get_rad_type_list
    rad_types = get_rad_type_list()

    if H_MR['type'] not in rad_types:
        return False

    if H_OR is None:
        return True

    if H_OR['type'] in rad_types:
        return True

    return False

def get_virtual_heatsource(region, H_HS):
    default_HS = get_default_heatsource(region)

    if H_HS is None:
        return default_HS
    elif H_HS['type'] == 'その他の温水暖房機':
        return {
            'type': default_HS['type'],
            'e_rtd_hs': default_HS['e_rtd_hs'],
            'pipe_insulation': H_HS['pipe_insulation'],
            'underfloor_pipe_insulation': H_HS['underfloor_pipe_insulation'],
        }
    elif H_HS['type'] == '温水暖房機を設置しない':
        return {
            'type': default_HS['type'],
            'e_rtd_hs': default_HS['e_rtd_hs'],
            'pipe_insulation': False,
            'underfloor_pipe_insulation': False,
        }
    else:
        return H_HS


# 暖房区画1つと1つの暖房設備機器によって消費される電力消費量
def calc_E_E_H_d_t(i, device, A_A, A_MR, A_OR, region, mode, L_H_d_t):
    # 床面積の取得
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':

        # 仕様の取得
        q_rtd_C = rac_spec.get_q_rtd_C(A_HCZ)
        q_rtd_H = rac_spec.get_q_rtd_H(q_rtd_C)
        e_rtd_C = rac_spec.get_e_rtd_C(device['e_class'], q_rtd_C)
        e_rtd_H = rac_spec.get_e_rtd_H(e_rtd_C)

        # 消費電力量の計算
        E_E_H_d_t = rac.get_E_E_H_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            dualcompressor=device['dualcompressor'],
            L_H_d_t=L_H_d_t,
        )

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        q_min_H = ff_spec.get_q_min_H(q_max_H)
        P_rtd_H = ff_spec.get_P_rtd_H(q_max_H)
        P_itm_H = ff_spec.get_P_itm_H()

        # 消費電力量の計算
        E_E_H_d_t = ff.get_E_E_H_d_t(
            q_max_H=q_max_H,
            q_min_H=q_min_H,
            P_rtd_H=P_rtd_H,
            P_itm_H=P_itm_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気ヒーター床暖房':

        # 仕様の取得
        r_Af = device['r_Af']
        r_up = device['r_up']
        A_f = eheater_spec.get_A_f(A_HCZ, r_Af)

        # 消費電力量の計算
        E_E_H_d_t = eheater.get_E_E_H_d_t(
            A_f=A_f,
            r_up=r_up,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':

        # 仕様の取得
        q_rq_H = ets_spec.get_q_rq_H(region)
        f_cT = ets_spec.get_f_cT(region)
        f_cI = ets_spec.get_f_cI(mode, '主たる居室' if i == 1 else 'その他の居室')
        q_rtd_H = ets_spec.get_q_rtd_H(q_rq_H, A_HCZ, f_cT, f_cI)
        e_rtd_H = ets_spec.get_e_rtd_H()

        # 消費電力量の計算
        E_E_H_d_t = ets.get_E_E_H_d_t(
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':

        # 仕様の取得
        r_Af = device['r_Af']
        r_up = device['r_up']
        pipe_insulation = device['pipe_insulation']

        # 消費電力量の計算
        E_E_H_d_t = racfh.get_E_E_d_t(
            region=region,
            A_A_act=A_A,
            i=i,
            A_HCZ=A_HCZ,
            r_Af=r_Af,
            r_up=r_up,
            pipe_insulation=pipe_insulation,
            L_H_d_t=L_H_d_t
        )

    else:
        raise ValueError(device['type'])

    return E_E_H_d_t


def get_L_H_d_t(region, sol_region, A_A, A_MR, A_OR, mode_H, H_MR, H_OR, mode_MR, mode_OR, Q, mu_H, TS, r_A_ufvnt, HEX,
                SHC, underfloor_insulation, normalize=True):
    # 温水暖房
    floorheating_i = get_floorheating_i(H_MR, H_OR)
    R_l_i = get_R_l_i(H_MR, H_OR)

    # 運転モード
    if mode_H == '住戸全体を連続的に暖房する方式':
        # 全館連続
        mode = tuple(["全館連続"] * 12)
    else:
        mode = (mode_MR, None, mode_OR, mode_OR, mode_OR, None, None, None, None, None, None, None)

    # 暖冷房区画i݅の1時間当たりの暖房負荷 (1)
    args = {
        'region': region,
        'A_A': A_A,
        'A_MR': A_MR,
        'A_OR': A_OR,
        'mode': mode,

        # 外皮
        'Q': Q,
        'mu_H': mu_H,
        'TS': TS,

        # 床下換気
        'r_A_ufvnt': r_A_ufvnt,
        'underfloor_insulation': underfloor_insulation,

        # 床下暖房
        'floorheating': tuple(floorheating_i),
        'R_l_i': tuple(R_l_i),
    }

    # 熱交換型換気
    if HEX is not None:
        args['hex'] = True
        args['etr_dash_t'] = calc_etr_dash_t(
            etr_t_raw=HEX['etr_t'],
            e=HEX['e'],
            C_bal=HEX['C_bal'],
            C_leak=HEX['C_leak']
        )
    else:
        args['hex'] = False
        args['etr_dash_t'] = None

    # 太陽熱集熱式
    if SHC is not None and SHC['type'] == '空気集熱式':
        args.update({
            'hotwater_use': SHC['hotwater_use'],
            'supply_target': SHC['supply_target'],
            'sol_region': sol_region,
            'P_alpha': SHC['P_alpha'],
            'P_beta': SHC['P_beta'],
            'A_col': SHC['A_col'],
            'V_fan_P0': SHC['V_fan_P0'],
            'm_fan_test': SHC['m_fan_test'],
            'd0': SHC['d0'],
            'd1': SHC['d1'],
            'ufv_insulation': SHC['ufv_insulation'],
            'r_A_ufvnt_ass': SHC['r_A_ufvnt_ass'],
        })

    L_T_H_d_t_i, L_dash_H_R_d_t_i = ld.get_L_H_d_t_i(**args)

    if normalize:
        L_T_H_d_t_i[L_T_H_d_t_i < 0] = 0

    return L_T_H_d_t_i, L_dash_H_R_d_t_i


def get_floorheating_i(H_MR, H_OR):
    floorheating = np.zeros(12)

    if H_MR is not None:
        if H_MR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            floorheating[0] = True

    if H_OR is not None:
        if H_OR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            floorheating[1:6] = True

    return floorheating


def get_R_l_i(H_MR, H_OR):
    # 床暖房
    R_l_i = np.zeros(12)

    if H_MR is not None:
        if H_MR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            if 'r_dash_Af' not in H_MR:
                R_l_i[0] = H_MR['r_Af']
            else:
                # 吹き抜けを有する場合
                R_l_i[0] = H_MR['r_dash_Af']

    if H_OR is not None:
        if H_OR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            R_l_i[1:6] = H_OR['r_Af']

    return R_l_i


def get_A_HCZ_i(i, A_A, A_MR, A_OR):
    return ld.get_A_HCZ_i(i, A_A, A_MR, A_OR)


# 暖房設備のガス消費量（MJ/h）
def get_E_G_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t_i):
    return get_E_G_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 暖房設備機器等のガス消費量（MJ/h）
def get_E_G_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i, CG):
    if H_A is not None:
        # (7b)
        return get_E_G_H_hs_A_d_t()
    elif (spec_MR is not None or spec_OR is not None) and L_T_H_d_t_i is not None:
        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合
            return get_E_G_H_hs_MROR_d_t(mode_MR, mode_OR, spec_HS, spec_MR, spec_OR, L_T_H_d_t_i, CG, A_A, A_MR, A_OR,
                                         region)
        else:
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合
            return get_E_G_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                       L_T_H_d_t_i, CG) \
                   + get_E_G_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                         L_T_H_d_t_i, CG)
    else:
        return np.zeros(24*365)


# 暖房区画1つと1つの暖房設備機器によって消費されるガス消費量
def calc_E_G_H_d_t(i, device, A_A, A_MR, A_OR, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':
        # ガス消費量の計算
        E_G_H_d_t = rac.get_E_G_H_d_t()

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        if 'e_rtd_H' in device:
            e_rtd_H = device['e_rtd_H']
        else:
            e_rtd_H = ff_spec.get_e_rtd_H_default()

        # ガス消費量の計算
        E_G_H_d_t = ff.get_E_G_H_d_t(
            fuel='K',
            q_max_H=q_max_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':
        # ガス消費量の計算
        E_G_H_d_t = ets.get_E_G_H_d_t()

    elif device['type'] == '電気ヒーター床暖房':
        # ガス消費量の計算
        E_G_H_d_t = eheater.get_E_G_H_d_t()

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':
        # ガス消費量の計算
        E_G_H_d_t = racfh.get_E_G_d_t()

    else:
        raise ValueError(device['type'])

    return E_G_H_d_t


# 暖房設備の灯油消費量（MJ/h）
def get_E_K_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t_i):
    return get_E_K_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i,
                          CG)


# 暖房設備機器等の灯油消費量（MJ/h）
def get_E_K_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i, CG):
    if H_A is not None:
        # (7c)
        return get_E_K_H_hs_A_d_t()
    elif (spec_MR is not None or spec_OR is not None) and L_T_H_d_t_i is not None:
        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合
            return get_E_K_H_hs_MROR_d_t(spec_HS, spec_MR, spec_OR, CG, A_A, A_MR, A_OR, region, mode_MR, mode_OR,
                                         L_T_H_d_t_i)
        else:

            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合
            return get_E_K_H_hs_MR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR,
                                       L_T_H_d_t_i, CG) \
                   + get_E_K_H_hs_OR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR,
                                         L_T_H_d_t_i, CG)
    else:
        return np.zeros(24*365)

# 実質的な暖房機器の仕様を取得
def get_virtual_heating_devices(region, H_MR, H_OR):
    # 規定の暖房機器を取得
    default_spec = get_default_heating_spec(region)

    # 設置しないまたはその他の機器の場合は規定の機器に置き換える
    if H_MR is not None:
        if H_MR['type'] == '設置しない' or H_MR['type'] == 'その他':
            H_MR = default_spec[0]
    if H_OR is not None:
        if H_OR['type'] == '設置しない' or H_OR['type'] == 'その他':
            H_OR = default_spec[1]

    return H_MR, H_OR


# 暖房区画1つと1つの暖房設備機器によって消費されるガス消費量
def calc_E_K_d_t(i, device, A_A, A_MR, A_OR, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':
        # 灯油の消費量の計算
        E_K_d_t = rac.get_E_K_H_d_t()

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        if 'e_rtd_H' in device:
            e_rtd_H = device['e_rtd_H']
        else:
            e_rtd_H = ff_spec.get_e_rtd_H_default()

        # ガス消費量の計算
        E_K_d_t = ff.get_E_K_H_d_t(
            fuel='K',
            q_max_H=q_max_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':
        # 灯油の消費量の計算
        E_K_d_t = ets.get_E_K_H_d_t()

    elif device['type'] == '電気ヒーター床暖房':
        # 灯油の消費量の計算
        E_K_d_t = eheater.get_E_K_H_d_t()

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':
        # 灯油の消費量の計算
        E_K_d_t = racfh.get_E_K_d_t()

    else:
        raise ValueError(device['type'])

    return E_K_d_t


# 暖房設備のその他の燃料による一次エネルギー消費量（MJ/h）(6d)
def get_E_M_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i):
    return get_E_M_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i)


# 暖房設備機器等のその他の燃料による一次エネルギー消費量（MJ/h）(7d,8d,9d)
def get_E_M_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, L_H_d_t):
    if H_A is not None:
        # (7d)
        return get_E_M_H_hs_A_d_t()
    else:

        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合
            # (8d)
            return get_E_M_H_hs_MROR_d_t(region, spec_HS)
        else:
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合
            # (9d)
            return get_E_M_H_hs_MR_d_t(A_A, A_MR, A_OR, spec_MR, spec_HS, L_H_d_t) + \
                   get_E_M_H_hs_OR_d_t(A_A, A_MR, A_OR, spec_OR, spec_HS, L_H_d_t)


#### 6.3 暖房設備機器のエネルギー消費量 / Energy consumption of heating equipment

#### 6.3.1 住戸全体を連続的に暖房する方式 / Method to heat the whole dwelling unit continuously

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,A,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,A,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,A,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,A,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h) / Kerosene consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of heating installation equipment per hour at time $ t $ of date $ d $  
$E_{E,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ on date $ d $ in the method of heating the entire dwelling unit continuously  
$E_{G,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ of date $ d $ in the scheme of heating the entire dwelling unit continuously  
$E_{K,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h) / Kerosene consumption of heating equipment per hour at time $ t $ of date $ d $ in the scheme of heating the entire dwelling unit continuously  
$E_{M,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of heating equipment and equipment per hour at time $ t $ on date $ d $ in the scheme of heating the entire dwelling unit continuously  

In [ ]:
# 1時間当たりの暖房設備機器の消費電力量
def get_E_E_H_hs_A_d_t(region, A_A, H_A, L_H_A_d_t):
    # 仕様の取得
    if 'q_rtd_H' in H_A and 'P_rtd_H' in H_A:
        q_rtd_H = H_A['q_rtd_H']
        P_rtd_H = H_A['P_rtd_H']
    else:
        q_rtd_H = dc_spec.get_q_rtd_H(region, A_A)
        P_rtd_H = dc_spec.get_P_rtd_H(q_rtd_H)
    C_pl_H = H_A['C_pl_H']

    # 電力消費量の計算
    E_E_H_d_t = dc.get_E_E_H_d_t(
        region=region,
        q_rtd_H=q_rtd_H,
        P_rtd_H=P_rtd_H,
        C_pl_H=C_pl_H,
        L_H_d_t=L_H_A_d_t
    )

    return E_E_H_d_t


# 1時間当たりの暖房設備機器のガス電力量
def get_E_G_H_hs_A_d_t():
    return dc.get_E_G_H_d_t()


# 1時間当たりの暖房設備機器の灯油電力量
def get_E_K_H_hs_A_d_t(**args):
    return dc.get_E_K_H_d_t()


# 1時間当たりの暖房設備機器のその他の一次エネルギー消費量
def get_E_M_H_hs_A_d_t():
    return dc.get_E_M_H_d_t()

#### 6.3.2 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合 / In the case of heating only the room and installing hot water heating in the main room and other rooms

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,MROR,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,MROR,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,MROR,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,MROR,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h) / Kerosene consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of heating installation equipment per hour at time $ t $ of date $ d $  
$E_{E,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備の消費電力量(kWh/h) / Power consumption of heating facilities installed in the main room and other rooms per hour at the time $ t $ of date $ d $ in the method of heating only the room  
$E_{G,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備のガス消費量(MJ/h) / Gas consumption of heating facilities installed in the main room and other rooms per hour at the time $ t $ of date $ d $ in the method of heating only the room  
$E_{K,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備の灯油消費量(MJ/h) / Kerosene consumption of heating facilities installed in the main room and other rooms per hour at the time $ t $ of date $ d $ in the method of heating only the room  
$E_{M,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by the other fuels of the heating facilities installed in the main living room and the other living rooms per hour at the time $ t $ of date $ d $ in the method of heating only the living room  

In [ ]:
# 1時間当たりの暖房設備機器の消費電力量
def get_E_E_H_hs_MROR_d_t(region, A_A, A_MR, A_OR, H_MR, H_OR, H_HS, CG, L_T_H_d_t_i):
    # 暖房方式及び運転方法の区分
    mode_MR, mode_OR = get_heating_mode(region=region, H_MR=H_MR, H_OR=H_OR)

    return hwh.get_E_E_H_d_t(H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 1時間当たりの暖房設備機器のガス電力量
def get_E_G_H_hs_MROR_d_t(mode_MR, mode_OR, spec_HS, spec_MR, spec_OR, L_T_H_d_t_i, CG, A_A, A_MR, A_OR, region):
    return hwh.get_E_G_H_d_t(spec_HS, spec_MR, spec_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 1時間当たりの暖房設備機器の灯油電力量
def get_E_K_H_hs_MROR_d_t(spec_HS, spec_MR, spec_OR, CG, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i):
    return hwh.get_E_K_H_d_t(spec_HS, spec_MR, spec_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 1時間当たりの暖房設備機器のその他の一次エネルギー消費量
def get_E_M_H_hs_MROR_d_t(region, H_HS):
    # 実質的な温水暖房熱源の取得
    spec_HS = get_virtual_heatsource(region, H_HS)

    return hwh.get_E_M_H_d_t(spec_HS)

#### 6.3.3 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合 / In the case that it is a method to heat only a living room and hot water heating is installed in the main living room and other living rooms 

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,MR,d,t} + E_{E,H,hs,OR,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,MR,d,t} + E_{G,H,hs,OR,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,MR,d,t} + E_{K,H,hs,OR,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,MR,d,t} + E_{M,H,hs,OR,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ on date $ d $ (kWh / h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ on date $ d $ (MJ / h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h) / Kerosene consumption of heating equipment per hour at time $ t $ on date $ d $ (MJ / h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by heating system equipment's other fuel per hour at time $ t $ on date $ d $ (MJ / h)  
$E_{E,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器の消費電力量(kWh/h) / power consumption of heating equipment installed in the main room per hour at time $ t $ of date $ d $ in the method of heating only the room Amount (kWh / h)  
$E_{G,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器のガス消費量(MJ/h) / Gas consumption of heating equipment installed in the main living room per hour at time $ t $ on date $ d $ in the method of heating only the living room Amount (MJ / h)  
$E_{K,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器の灯油消費量(MJ/h) / Kerosene consumption of heating equipment installed in the main room per hour at time $ t $ on date $ d $ in the method of heating only the room Amount (MJ / h)  
$E_{M,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Other heating equipment installed in the main living room per hour at the time $ t $ dated $ d $ in the method of heating only the living room Primary energy consumption by fuel (MJ / h)  
$E_{E,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器の消費電力量(kWh/h) / Consumption of heating equipment installed in other rooms per hour at time $ t $ on date $ d $ in the method of heating only the room Electric energy (kWh / h)  
$E_{G,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器のガス消費量(MJ/h) / Gas of heating equipment installed in other rooms per hour at time $ t $ on date $ d $ in the method of heating only the room Consumption (MJ / h)  
$E_{K,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器の灯油消費量(MJ/h) / Kerosene of heating equipment installed in other rooms per hour at time $ t $ on date $ d $ in the method of heating only the room Consumption (MJ / h)  
$E_{M,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Other heating installation equipment installed in other rooms per hour at time $ t $ on date $ d $ in the method of heating only the room Primary energy consumption by fuel (MJ / h)  

In [ ]:
def get_E_E_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i, CG=None):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_E_H_d_t(
            H_HS=spec_HS,
            H_MR=spec_MR,
            H_OR=spec_OR,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            region=region,
            mode_MR=mode_MR,
            mode_OR=mode_OR,
            L_T_H_rad=L_T_H_d_t_i,
            CG=CG)
    else:
        return calc_E_E_H_d_t(1, spec_MR, A_A, A_MR, A_OR, region, mode_MR, L_T_H_d_t_i[0])
    
def get_E_G_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t, CG):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_G_H_d_t(
            H_HS=spec_HS,
            H_MR=spec_MR,
            H_OR=spec_OR,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            region=region,
            mode_MR=mode_MR,
            mode_OR=mode_OR,
            L_T_H_rad=L_T_H_d_t,
            CG=CG)
    else:

        return calc_E_G_H_d_t(1, spec_MR, A_A, A_MR, A_OR, L_T_H_d_t[0])
    
# 居室のみを暖房する方式における日付の時刻における 1 時間当たりの主たる居室に設置された暖房設備機器のガス消費量
def get_E_K_H_hs_MR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t, CG):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_K_H_d_t(
            H_HS=spec_HS,
            H_MR=spec_MR,
            H_OR=spec_OR,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            region=region,
            mode_MR=mode_MR,
            mode_OR=mode_OR,
            L_T_H_rad=L_T_H_d_t,
            CG=CG)
    else:
        return calc_E_K_d_t(1, spec_MR, A_A, A_MR, A_OR, L_T_H_d_t[0])

# 居室のみを暖房する方式における日付の時刻における 1 時間当たりの主たる居室に設置された暖房設備機器のその他の一次エネルギー
def get_E_M_H_hs_MR_d_t(A_A, A_MR, A_OR, spec_MR, spec_HS, L_H_d_t):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_M_H_d_t(spec_HS)
    else:
        return calc_E_M_d_t(1, spec_MR, A_A, A_MR, A_OR, L_H_d_t[0])

$$
\large
E_{E,H,hs,MR,d,t} = E_{E,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{G,H,hs,MR,d,t} = E_{G,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{K,H,hs,MR,d,t} = E_{K,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{M,H,hs,MR,d,t} = E_{M,H,hs,d,t,i}|_{i=1}
$$

$E_{E,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器の消費電力量(kWh/h) / Power consumption of heating equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$E_{G,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器のガス消費量(MJ/h) / Gas consumption of heating equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$E_{K,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器の灯油消費量(MJ/h) / Kerosene consumption of heating equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$E_{M,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of heating equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  

$$
\large
E_{E,H,hs,OR,d,t} = \sum_{i=2～5} E_{E,H,hs,d,t,i} 
$$

$$
\large
E_{G,H,hs,OR,d,t} = \sum_{i=2～5} E_{G,H,hs,d,t,i} 
$$

$$
\large
E_{K,H,hs,OR,d,t} = \sum_{i=2～5} E_{K,H,hs,d,t,i} 
$$

$$
\large
E_{M,H,hs,OR,d,t} = \sum_{i=2～5} E_{M,H,hs,d,t,i} 
$$

In [ ]:
def get_E_E_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i, CG=None):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)
    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_E_H_d_t(
                H_HS=spec_HS,
                H_MR=spec_MR,
                H_OR=spec_OR,
                A_A=A_A,
                A_MR=A_MR,
                A_OR=A_OR,
                region=region,
                mode_MR=mode_MR,
                mode_OR=mode_OR,
                L_T_H_rad=L_T_H_d_t_i,
                CG=CG)
        else:
            # 暖房区画i=2～5の電力消費量を計算して合算する
            return np.sum(
                [calc_E_E_H_d_t(i, spec_OR, A_A, A_MR, A_OR, region, mode_OR, L_T_H_d_t_i[i - 1]) for i in
                 [3, 4, 5]], axis=0)

def get_E_G_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t, CG=None):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)

    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_G_H_d_t(
                H_HS=spec_HS,
                H_MR=spec_MR,
                H_OR=spec_OR,
                A_A=A_A,
                A_MR=A_MR,
                A_OR=A_OR,
                region=region,
                mode_MR=mode_MR,
                mode_OR=mode_OR,
                L_T_H_rad=L_T_H_d_t,
                CG=CG)
        else:
            return np.sum(
                [calc_E_G_H_d_t(i, spec_OR, A_A, A_MR, A_OR, L_T_H_d_t[i - 1])
                 for i in
                 range(2, 6)], axis=0)

# 居室のみを暖房する方式における日付の時刻における 1 時間当たりのその他の居室に設置された暖房設備機器のガス消費量
def get_E_K_H_hs_OR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t, CG):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)

    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_K_H_d_t(
                H_HS=spec_HS,
                H_MR=spec_MR,
                H_OR=spec_OR,
                A_A=A_A,
                A_MR=A_MR,
                A_OR=A_OR,
                region=region,
                mode_MR=mode_MR,
                mode_OR=mode_OR,
                L_T_H_rad=L_T_H_d_t,
                CG=CG)
        else:
            return np.sum(
                [calc_E_K_d_t(i, spec_OR, A_A, A_MR, A_OR, L_T_H_d_t[i - 1])
                 for i in
                 range(2, 6)], axis=0)

# 居室のみを暖房する方式における日付の時刻における 1 時間当たりのその他の居室に設置された暖房設備機器のその他の一次エネルギー
def get_E_M_H_hs_OR_d_t(A_A, A_MR, A_OR, spec_OR, spec_HS, L_H_d_t):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)

    else:

        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_M_H_d_t(spec_HS)
        else:
            return np.sum(
                [calc_E_M_d_t(i, spec_OR, A_A, A_MR, A_OR, L_H_d_t[i - 1]) for
                 i in
                 range(2, 6)], axis=0)


# 暖房区画1つと1つの暖房設備機器によって消費されるその他の一次エネルギー消費量
def calc_E_M_d_t(i, device, A_A, A_MR, A_OR, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = rac.get_E_M_H_d_t()

    elif device['type'] == 'FF暖房機':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = ff.get_E_M_H_d_t()

    elif device['type'] == '電気蓄熱暖房器':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = ets.get_E_M_H_d_t()

    elif device['type'] == '電気ヒーター床暖房':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = eheater.get_E_M_H_d_t()

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = racfh.get_E_M_d_t()

    else:
        raise ValueError(device['type'])

    return E_M_d_t

#### 6.4 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値 / Design primary energy consumption equivalent value of un-processed heating load of heating installation

#### 6.4.1 住戸全体を連続的に暖房する方式 / Method to heat the whole dwelling unit continuously

$$
\large
E_{UT,H,d,t} = Q_{UT,H,A,d,t} \times \alpha_{UT,H,A}
$$

$E_{UT,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h) / Design Primary Energy Consumption Equivalents of Raw Heating Load of Heating Equipment Per Hour at Time $ t $ of Date $ d $  
$Q_{UT,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷(MJ/h) / Raw heating load of heating equipment per hour at time $ t $ of date $ d $ in the scheme of heating the whole dwelling unit continuously  
$\alpha_{UT,H,A}$：住戸全体を連続的に暖房する方式における暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数 / A factor to convert the raw heating load of the heating equipment in the method of heating the entire dwelling unit continuously to the design primary energy consumption equivalent value of the raw heating load  

In [ ]:
# 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値
def get_E_UT_H_d_t__modeA(region, A_A, H_A, L_H_A_d_t):
    # 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
    alpha_UT_H_A = get_alpha_UT_H_A(region)

    # 仕様の取得
    if 'q_rtd_H' in H_A and 'P_rtd_H' in H_A:
        q_rtd_H = H_A['q_rtd_H']
    else:
        q_rtd_H = dc_spec.get_q_rtd_H(region, A_A)

    # 未処理負荷を取得
    Q_UT_H_A_d_t = dc.get_Q_UT_H_d_t(
        region=region,
        q_rtd_H=q_rtd_H,
        L_H_d_t=L_H_A_d_t
    )

    return Q_UT_H_A_d_t * alpha_UT_H_A

<div style="text-align: center;font-weight: bold;">表　未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数<br>Table: Factors to convert the raw heating load into the design primary energy consumption equivalent value of the raw heating load</div>

|地域の区分<br>Segmentation of area|住戸全体を連続的に暖房する方式<br>Method to heat the whole dwelling unit continuously|
|-|-|
|1|1.61|
|2|1.46|
|3|1.32|
|4|1.30|
|5|1.20|
|6|1.09|
|7|1.12|
|8|‐|

In [ ]:
# 表 3 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
table_3 = [
    1.61,
    1.46,
    1.32,
    1.30,
    1.20,
    1.09,
    1.12
]

In [ ]:
# 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
def get_alpha_UT_H_A(region):
    return table_3[region - 1]

#### 6.4.2 居室のみを暖房する方式 / Method to heat only room

$$
\large
E_{UT,H,d,t} = Q_{UT,H,MR,d,t} \times \alpha_{UT,H,MR} + Q_{UT,H,OR,d,t} \times \alpha_{UT,H,OR}
$$

$E_{UT,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h) / Design Primary Energy Consumption Equivalents of Raw Heating Load of Heating Equipment Per Hour at Time $ t $ of Date $ d $  
$Q_{UT,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の未処理暖房負荷(MJ/h) / Raw heating load of the heating equipment installed in the main room per hour at the time $ t $ of date $ d $ in the method of heating only the room  
$Q_{UT,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の未処理暖房負荷(MJ/h) / Raw heating load of heating facilities installed in other rooms per hour at the time $ t $ of date $ d $ in the method of heating only the room    
$\alpha_{UT,H,MR}$：居室のみを暖房する方式における主たる居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数 / A factor to convert the unprocessed heating load of the heating equipment installed in the main room in the method of heating only the room to the design primary energy consumption equivalent value of the unprocessed heating load  
$\alpha_{UT,H,OR}$：居室のみを暖房する方式におけるその他の居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数 / A factor to convert the unprocessed heating load of the heating equipment installed in other rooms in the method of heating only the room to the design primary energy consumption equivalent value of the unprocessed heating load  

<div style="text-align: center;font-weight: bold;">表　未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数<br>Table: Factors to convert the raw heating load into the design primary energy consumption equivalent value of the raw heating load</div>

<table>
<thead>
<tr>
<th rowspan="3">地域の区分<br>Segmentation of area</th>
<th colspan="4">居室のみを暖房する方式<br>Method to heat only room</th>
</tr>
<tr>
<th colspan="2">主たる居室<br>Main room</th>
<th colspan="2">その他の居室<br>Other rooms</th>
</tr>
<tr>
<th>連続運転<br>Continuous operation</th>
<th>間歇運転<br>Intermittent operation</th>
<th>連続運転<br>Continuous operation</th>
<th>間歇運転<br>Intermittent operation</th>
</tr>
</thead>
<tbody>
<tr><th>1</th><td>1.59</td><td>1.21</td><td>1.59</td><td>1.22</td></tr>
<tr><th>2</th><td>1.66</td><td>1.22</td><td>1.66</td><td>1.24</td></tr>
<tr><th>3</th><td>1.63</td><td>1.22</td><td>1.63</td><td>1.23</td></tr>
<tr><th>4</th><td>1.60</td><td>1.21</td><td>1.60</td><td>1.23</td></tr>
<tr><th>5</th><td>1.53</td><td>1.05</td><td>1.53</td><td>1.04</td></tr>
<tr><th>6</th><td>1.57</td><td>0.96</td><td>1.57</td><td>1.00</td></tr>
<tr><th>7</th><td>1.63</td><td>1.01</td><td>1.63</td><td>1.34</td></tr>
<tr><th>8</th><td>-</td><td>-</td><td>-</td><td>-</td></tr>
</tbody>
</table>

In [ ]:
# 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
table_4 = [
    (1.59, 1.21, 1.59, 1.22),
    (1.66, 1.22, 1.66, 1.24),
    (1.63, 1.22, 1.63, 1.23),
    (1.60, 1.21, 1.60, 1.23),
    (1.53, 1.05, 1.53, 1.04),
    (1.57, 0.96, 1.57, 1.00),
    (1.63, 1.01, 1.63, 1.34)
]

In [ ]:
# 居室のみを暖房する方式における主たる居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
def get_alpha_UT_H_MR(region, mode_H_MR):
    if mode_H_MR in ['ろ', '連続運転']:
        return table_4[region - 1][0]
    elif mode_H_MR in ['は', '間歇運転']:
        return table_4[region - 1][1]
    else:
        raise ValueError(mode_H_MR)


# 居室のみを暖房する方式におけるその他の居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
def get_alpha_UT_H_OR(region, mode_H_OR):
    if mode_H_OR in ['ろ', '連続運転']:
        return table_4[region - 1][0]
    elif mode_H_OR in ['は', '間歇運転']:
        return table_4[region - 1][3]
    else:
        raise ValueError(mode_H_OR)


#### 6.5 コージェネレーション設備又は住棟セントラル暖房設備が賄う温水暖房の熱負荷等 / Heat load, etc. of hot water heating provided by cogeneration facilities or the central heating facility in a residential building

省略

Omission

### 7．冷房設備の一次エネルギー消費量及び処理負荷と未処理負荷 / Primary energy consumption and processing load and untreated load of cooling equipment

#### 7.1 処理負荷及び未処理負荷 / Processing load and unprocessed load

#### 7.1.1 住戸全体を連続的に冷房する方式 / A method for continuously cooling the entire dwelling unit

$$
\large
Q_{T,CS,A,d,t} = min(Q_{max,CS,d,t}, L_{CS,A,d,t})
$$

$$
\large
Q_{UT,CS,A,d,t} = L_{CS,A,d,t} - Q_{T,CS,A,d,t}
$$

$Q_{T,CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の処理冷房顕熱負荷(MJ/h) / Processing cooling sensible heat load of cooling system per hour at time $ t $ of date $ d $ in a method of continuously cooling the whole dwelling unit  
$Q_{UT,CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の未処理冷房顕熱負荷(MJ/h) / Raw cooling sensible heat load of cooling system per hour at time $ t $ of date $ d $ in the method of continuously cooling the whole dwelling unit  
$Q_{max,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の最大冷房顕熱出力(MJ/h) / Maximum cooling sensible heat output of cooling equipment per hour at time $ t $ on date $ d $  
$L_{CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷の合計(MJ/h) / Sum of cooling sensible heat load per hour at time $ t $ of date $ d $ in the method of continuously cooling the whole dwelling unit

In [ ]:
def get_Q_T_CS_A_d_t(Q_max_CS_d_t, L_CS_A_d_t):
    return np.min([Q_max_CS_d_t, L_CS_A_d_t], axis=0)


def get_Q_UT_CS_A_d_t(L_CS_A_d_t, Q_T_CS_A_d_t):
    return L_CS_A_d_t - Q_T_CS_A_d_t

$$
\large
Q_{T,CL,A,d,t} = min(Q_{max,CL,d,t}, L_{CL,A,d,t})
$$

$$
\large
Q_{UT,CL,A,d,t} = L_{CL,A,d,t} - Q_{T,CL,A,d,t}
$$

$Q_{T,CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の処理冷房潜熱負荷(MJ/h) / Processed cooling latent heat load of cooling system per hour at time $ t $ of date $ d $ in the method of continuously cooling the whole dwelling unit  
$Q_{UT,CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の未処理冷房潜熱負荷(MJ/h) / Raw cooling latent heat load of cooling system per hour at time $ t $ of date $ d $ in the method of continuously cooling the whole dwelling unit  
$Q_{max,C,d,tL}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の最大冷房潜熱出力(MJ/h) / Maximum cooling latent heat output of cooling equipment per hour at time $ t $ on date $ d $  
$L_{CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房潜熱負荷の合計(MJ/h) / Sum of cooling latent heat load per hour at time $ t $ of date $ d $ in the method of continuously cooling the whole dwelling unit

$L_{CS,A,d,t}$が0未満の場合は0とする。$L_{CL,A,d,t}$が0未満の場合は0とする。 

If $ L_ {CS, A, d, t} $ is less than 0, it is 0. If $ L_ {CL, A, d, t} $ is less than 0, it is 0.

In [ ]:
def get_Q_T_CL_A_d_t(Q_max_CL_d_t, L_CL_A_d_t):
    return np.min([Q_max_CL_d_t, L_CL_A_d_t], axis=0)


def get_Q_UT_CL_A_d_t(L_CL_A_d_t, Q_T_CL_A_d_t):
    return L_CL_A_d_t - Q_T_CL_A_d_t

$$
\large
L_{CS,A,d,t} = \sum_{i=1～12} L_{CS,d,t,i} 
$$

$$
\large
L_{CL,A,d,t} = \sum_{i=1～12} L_{CL,d,t,i} 
$$

$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房顕熱負荷(MJ/h) / Cooling sensible heat load per hour of heating / cooling division $ i $ at time $ t $ of date $ d $  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房潜熱負荷(MJ/h) / Cooling latent heat load per hour of heating / cooling section $ i $ at time $ t $ of date $ d $  

In [ ]:
# 住戸全体を連続的に冷房する方式における1時間当たりの冷房顕熱負荷の合計
def get_L_CS_A_d_t(region, A_A, A_MR, A_OR, Q, mu_C, NV_MR, NV_OR, r_A_ufvnt, underfloor_insulation):
    # i=1～12の冷房区画の冷房顕熱負荷を計算して合算
    mode = ['全館連続'] * 12
    L_CS_A_d_t = np.sum(ld.get_L_CS_d_t_i(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        mode=mode,
        NV_MR=NV_MR,
        NV_OR=NV_OR,
        Q=Q,
        mu_C=mu_C,
        r_A_ufvnt=r_A_ufvnt,
        underfloor_insulation=underfloor_insulation,
    ), axis=0)

    L_CS_A_d_t[L_CS_A_d_t < 0] = 0

    return L_CS_A_d_t


# 住戸全体を連続的に冷房する方式における1時間当たりの冷房潜熱負荷の合計
def get_L_CL_A_d_t(region, A_A, A_MR, A_OR, Q, mu_C, NV_MR, NV_OR):
    # i=1～12の冷房区画の冷房潜熱負荷を計算して合算
    mode = ['全館連続'] * 12
    L_CL_A_d_t = np.sum(ld.get_L_CL_d_t_i(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        mode=mode,
        NV_MR=NV_MR,
        NV_OR=NV_OR,
        Q=Q,
        mu_C=mu_C
    ), axis=0)

    L_CL_A_d_t[L_CL_A_d_t < 0] = 0

    return L_CL_A_d_t

#### 7.1.2 居室のみを冷房する方式 / Method of cooling only the room

$$
\large
Q_{T,CS,MR,d,t} = Q_{T,CS,d,t,i}|_{i=1}
$$

$$
\large
Q_{T,CL,MR,d,t} = Q_{T,CL,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,CS,MR,d,t} = Q_{UT,CS,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,CL,MR,d,t} = Q_{UT,CL,d,t,i}|_{i=1}
$$

$Q_{T,CS,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の処理冷房顕熱負荷(MJ/h) / Processing cooling sensible heat load of the cooling system installed in the main room per hour at time $ t $ of date $ d $ in the method of cooling only the room  
$Q_{T,CL,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の処理冷房潜熱負荷(MJ/h) / Processing cooling latent heat load of the cooling system installed in the main room per hour at the time $ t $ of date $ d $ in the method of cooling only the room MJ / h)  
$Q_{UT,CS,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の未処理冷房顕熱負荷(MJ/h) / Untreated cooling sensible heat of the cooling system installed in the main room per hour at the time $ t $ of date $ d $ in the method of cooling only the room Load  
$Q_{UT,CL,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の未処理冷房潜熱負荷(MJ/h) / Raw cooling latent heat load of the cooling system installed in the main room per hour at the time $ t $ of date $ d $ in the method of cooling only the room  
$Q_{T,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の処理冷房顕熱負荷(MJ/h) / Processing cooling sensible heat load (MJ of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$Q_{T,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の処理冷房潜熱負荷(MJ/h) / Processing cooling latent heat load of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$Q_{UT,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の未処理冷房顕熱負荷(MJ/h) / Raw cooling sensible heat load of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$Q_{UT,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の未処理冷房潜熱負荷(MJ/h)	 / Raw cooling latent heat load of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ on date $ d $  

In [ ]:
# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の処理冷房顕熱負荷
def get_Q_T_CS_MR_d_t(**args):
    # i=1の冷房区画の処理冷房顕熱負荷を計算して合算
    return get_Q_T_CS_d_t_i(i=1, **args)


# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の処理冷房潜熱負荷
def get_Q_T_CL_MR_d_t(**args):
    # i=1の冷房区画の処理冷房潜熱負荷を計算して合算
    return get_Q_T_CL_d_t_i(i=1, **args)


# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の未処理処理冷房顕熱負荷
def get_Q_UT_CS_MR_d_t(**args):
    # i=1の冷房区画の未処理冷房顕熱負荷を計算して合算
    return get_Q_UT_CS_d_t_i(i=1, **args)


# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の未処理処理冷房潜熱負荷
def get_Q_UT_CL_MR_d_t(**args):
    # i=1の冷房区画の未処理冷房潜熱負荷を計算して合算
    return get_Q_UT_CL_d_t_i(i=1, **args)

$$
\large
Q_{T,CS,OR,d,t} = \sum_{i=2～5} Q_{T,CS,d,t,i} 
$$

$$
\large
Q_{T,CL,OR,d,t} = \sum_{i=2～5} Q_{T,CL,d,t,i} 
$$

$$
\large
Q_{UT,CS,OR,d,t} = \sum_{i=2～5} Q_{UT,CS,d,t,i} 
$$

$$
\large
Q_{UT,CL,OR,d,t} = \sum_{i=2～5} Q_{UT,CL,d,t,i} 
$$

$Q_{T,CS,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の処理冷房顕熱負荷(MJ/h) / Processing cooling sensible heat load of cooling facilities installed in other rooms per hour at time $ t $ of date $ d $ in the method of cooling only the room  
$Q_{T,CL,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の処理冷房潜熱負荷(MJ/h) / Processing cooling latent heat load of cooling facilities installed in other rooms per hour at time $ t $ of date $ d $ in the method of cooling only the room  
$Q_{UT,CS,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の未処理冷房顕熱負荷(MJ/h) / Raw cooling sensible heat load of cooling facilities installed in other rooms per hour at time $ t $ of date $ d $ in the method of cooling only the room  
$Q_{UT,CL,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の未処理冷房潜熱負荷(MJ/h) / Raw cooling latent heat load of cooling equipment installed in other rooms per hour at time $ t $ of date $ d $ in the method of cooling only the room  

In [ ]:
# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の処理冷房顕熱負荷
def get_Q_T_CS_OR_d_t(**args):
    # i=2～5の冷房区画の処理冷房顕熱負荷を計算して合算
    return np.sum([get_Q_T_CS_d_t_i(i, **args) for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の処理冷房潜熱負荷
def get_Q_T_CL_OR_d_t(**args):
    # i=2～5の冷房区画の処理冷房潜熱負荷を計算して合算
    return np.sum([get_Q_T_CL_d_t_i(i, **args) for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の未処理冷房顕熱負荷
def get_Q_UT_CS_OR_d_t(**args):
    # i=2～5の冷房区画の未処理冷房顕熱負荷を計算して合算
    return np.sum([get_Q_UT_CS_d_t_i(i, **args) for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の未処理冷房潜熱負荷
def get_Q_UT_CL_OR_d_t(**args):
    # i=2～5の冷房区画の未処理冷房潜熱負荷を計算して合算
    return np.sum([get_Q_UT_CL_d_t_i(i, **args) for i in range(2, 6)], axis=0)

$$
\large
Q_{T,CS,d,t,i} = min(Q_{max,CS,d,t,i}, L_{CS,d,t,i})
$$

$$
\large
Q_{UT,CS,d,t,i} = L_{CS,d,t,i} - Q_{T,CS,d,t,i}
$$

$Q_{max,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の最大冷房顕熱出力(MJ/h) / Maximum cooling sensible heat output of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房顕熱負荷(MJ/h) / Cooling sensible heat load per hour of heating / cooling division $ i $ at time $ t $ of date $ d $  

In [ ]:
def get_Q_T_CS_d_t_i(Q_max_CS_d_t_i, L_CS_d_t_i):
    return np.min([Q_max_CS_d_t_i, L_CS_d_t_i], axis=0)


def get_Q_UT_CS_d_t_i(L_CS_d_t_i, Q_T_CS_d_t_i):
    return L_CS_d_t_i - Q_T_CS_d_t_i

$$
\large
Q_{T,CL,d,t,i} = min(Q_{max,CL,d,t,i}, L_{CL,d,t,i})
$$

$$
\large
Q_{UT,CL,d,t,i} = L_{CL,d,t,i} - Q_{T,CL,d,t,i}
$$

$Q_{max,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の最大冷房潜熱出力(MJ/h) / Maximum cooling latent heat output of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房潜熱負荷(MJ/h) / Cooling latent heat load per hour of heating / cooling section $ i $ at time $ t $ of date $ d $  

$L_{CS,d,t,i}$が0未満の場合は0とする。また、$L_{CL,d,t,i}$が0未満の場合は0とする。 

If $ L_ {CS, d, t, i} $ is less than 0, it is 0. Also, if $ L_ {CL, d, t, i} $ is less than 0, it is 0.

In [ ]:
def get_Q_T_CL_d_t_i(Q_max_CL_d_t_i, L_CL_d_t_i):
    return np.min([Q_max_CL_d_t_i, L_CL_d_t_i], axis=0)


def get_Q_UT_CL_d_t_i(L_CL_d_t_i, Q_T_CL_d_t_i):
    return L_CL_d_t_i - Q_T_CL_d_t_i

#### 7.2 冷房設備のエネルギー消費量 / Energy consumption of cooling system

$$
\large
E_{E,C,d,t} = E_{E,C,hs,d,t}
$$

$$
\large
E_{G,C,d,t} = E_{G,C,hs,d,t}
$$

$$
\large
E_{K,C,d,t} = E_{K,C,hs,d,t}
$$

$$
\large
E_{M,C,d,t} = E_{M,C,hs,d,t}
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の消費電力量(kWh/h) / Power consumption of cooling system per hour at time $ t $ of date $ d $   
$E_{G,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のガス消費量(MJ/h) / Gas consumption of cooling system per hour at time $ t $ on date $ d $  
$E_{K,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の灯油消費量(MJ/h) / Kerosene consumption of cooling system per hour at time $ t $ of date $ d $  
$E_{M,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by cooling system other fuel per hour at time $ t $ on date $ d $  
$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h) / Power consumption of cooling equipment per hour at time $ t $ of date $ d $   
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h) / Gas consumption of cooling equipment per hour at time $ t $ on date $ d $  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h) / Kerosene consumption of cooling equipment per hour at time $ t $ of date $ d $  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by cooling system equipment's other fuel per hour at time $ t $ on date $ d $  

In [ ]:
# 1時間当たりの冷房設備の消費電力量
def get_E_E_C_d_t(region, A_A, A_MR, A_OR, C_A=None, C_MR=None, C_OR=None, L_CS_A_d_t=None, L_CL_A_d_t=None,
                  L_CS_d_t=None, L_CL_d_t=None):
    return get_E_E_C_hs_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t)


# 1時間当たりの冷房設備のガス消費量
def get_E_G_C_d_t(**args):
    return get_E_G_C_hs_d_t()


# 1時間当たりの冷房設備の灯油消費量
def get_E_K_C_d_t(**args):
    return get_E_K_C_hs_d_t()


# 1時間当たりの冷房設備のその他の一次エネルギー消費量
def get_E_M_C_d_t(**args):
    return get_E_M_C_hs_d_t()

#### 7.3 冷房設備機器のエネルギー消費量 / Energy consumption of cooling equipment

#### 7.3.1 住戸全体を連続的に冷房する方式 / A method for continuously cooling the entire dwelling unit

$$
\large
E_{E,C,hs,d,t} = E_{E,C,hs,A,d,t}
$$

$$
\large
E_{G,C,hs,d,t} = E_{G,C,hs,A,d,t}
$$

$$
\large
E_{K,C,hs,d,t} = E_{K,C,hs,A,d,t}
$$

$$
\large
E_{M,C,hs,d,t} = E_{M,C,hs,A,d,t}
$$

$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h) / Power consumption of cooling equipment per hour at time $ t $ of date $ d $  
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h) / Air conditioning equipment gas consumption per hour at time $ t $ on date $ d $  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h) / Kerosene consumption of cooling equipment per hour at time $ t $ on date $ d $  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption of cooling system equipment by other fuel per hour at time $ t $ of date $ d $  
$E_{E,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h) / Power consumption of cooling equipment per hour at time $ t $ of date $ d $ in the method of continuously cooling the entire dwelling unit  
$E_{G,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h) / Gas consumption of cooling equipment per hour at time $ t $ of date $ d $ in the method of continuously cooling the whole dwelling unit  
$E_{K,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h) / Kerosene consumption of cooling equipment per hour at time $ t $ of date $ d $ in the method of continuously cooling the entire dwelling unit  
$E_{M,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of cooling equipment per hour at time $ t $ on date $ d $ in a system that continuously cools the entire dwelling unit  

In [ ]:
# 1時間当たりの冷房設備機器の消費電力量
def get_E_E_C_hs_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t):
    if C_A is not None:
        # 仕様の取得
        if 'q_rtd_C' in C_A and 'P_rtd_C' in C_A:
            q_rtd_C = C_A['q_rtd_C']
            P_rtd_C = C_A['P_rtd_C']
        else:
            q_rtd_C = dc_spec.get_q_rtd_C(region, A_A)
            P_rtd_C = dc_spec.get_P_rtd_C(q_rtd_C)
        C_pl_C = C_A['C_pl_C']

        # 消費電力量の計算
        E_E_C_d_t_i = dc.get_E_E_C_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            P_rtd_C=P_rtd_C,
            C_pl_C=C_pl_C,
            L_CS_d_t=L_CS_A_d_t,
            L_CL_d_t=L_CL_A_d_t
        )

        return E_E_C_d_t_i
    elif C_MR is not None or C_OR is not None:
        # 消費電力量の計算
        return get_E_E_C_hs_MR_d_t(region, A_A, A_MR, A_OR, C_MR, L_CS_d_t, L_CL_d_t) \
               + get_E_E_C_hs_OR_d_t(region, A_A, A_MR, A_OR, C_OR, L_CS_d_t, L_CL_d_t)
    else:
        return np.zeros(24*365)


def get_E_G_C_hs_d_t():
    return dc.get_E_G_C_hs_A_d_t()


def get_E_K_C_hs_d_t():
    return dc.get_E_K_C_hs_A_d_t()


def get_E_M_C_hs_d_t():
    return dc.get_E_M_C_hs_A_d_t()

#### 7.3.2 居室のみを冷房する方式 / Method of cooling only the room

$$
\large
E_{E,C,hs,d,t} = E_{E,C,hs,MR,d,t} +  E_{E,C,hs,OR,d,t}
$$

$$
\large
E_{G,C,hs,d,t} = E_{G,C,hs,MR,d,t} +  E_{G,C,hs,OR,d,t}
$$

$$
\large
E_{K,C,hs,d,t} = E_{K,C,hs,MR,d,t} +  E_{K,C,hs,OR,d,t}
$$

$$
\large
E_{M,C,hs,d,t} = E_{M,C,hs,MR,d,t} +  E_{M,C,hs,OR,d,t}
$$

$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h) / Power consumption of cooling equipment per hour at time $ t $ of date $ d $   
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h) / Gas consumption of cooling equipment per hour at time $ t $ on date $ d $   
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h) / Kerosene consumption of cooling equipment per hour at time $ t $ of date $ d $   
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by cooling system equipment's other fuel per hour at time $ t $ on date $ d $   
$E_{E,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器の消費電力量(kWh/h) / power consumption of cooling equipment installed in the main living room per hour at time $ t $ of date $ d $ in the method of cooling only the living room Amount   
$E_{G,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器のガス消費量(MJ/h) / Gas consumption of cooling equipment installed in the main living room per hour at time $ t $ of date $ d $ in the method of cooling only the living room Amount   
$E_{K,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器の灯油消費量(MJ/h) / Kerosene consumption of cooling equipment installed in the main living room per hour at time $ t $ of date $ d $ in the method of cooling only the living room Amount   
$E_{M,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / other cooling equipment installed in the main living room per hour at the time $ t $ of date $ d $ in the system for cooling only the living room Primary energy consumption by fuel   
$E_{E,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器の消費電力量(kWh/h) / Consumption of cooling equipment installed in other rooms per hour at time $ t $ on date $ d $ in the method of cooling only the room Electric energy   
$E_{G,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器のガス消費量(MJ/h) / gas of cooling equipment installed in other rooms per hour at time $ t $ of date $ d $ in the method of cooling only the room Consumption   
$E_{K,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器の灯油消費量(MJ/h) / kerosene of cooling equipment installed in other rooms per hour at time $ t $ of date $ d $ in the method of cooling only the room Consumption   
$E_{M,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Other cooling equipment installed in other rooms per hour at time $ t $ on date $ d $ in the method of cooling only the room Primary energy consumption by fuel   


In [ ]:
def get_E_G_C_hs_d_t():
    return get_E_G_C_hs_MR_d_t() + get_E_G_C_hs_OR_d_t()


def get_E_K_C_hs_d_t():
    return get_E_K_C_hs_MR_d_t() + get_E_K_C_hs_OR_d_t()


def get_E_M_C_hs_d_t():
    return get_E_M_C_hs_MR_d_t() + get_E_M_C_hs_OR_d_t()

$$
\large
E_{E,C,hs,MR,d,t} = E_{E,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{G,C,hs,MR,d,t} = E_{G,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{K,C,hs,MR,d,t} = E_{K,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{M,C,hs,MR,d,t} = E_{M,C,hs,d,t,i}|_{i=1}
$$

$E_{E,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の消費電力量(kWh/h) / Power consumption of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$E_{G,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器のガス消費量(MJ/h) / Gas consumption of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ of date $ d $  
$E_{K,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の灯油消費量(MJ/h) / Kerosene consumption of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ on date $ d $  
$E_{M,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of cooling equipment installed in heating / cooling section $ i $ per hour at time $ t $ on date $ d $  

In [ ]:
# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器の消費電力
def get_E_E_C_hs_MR_d_t(region, A_A, A_MR, A_OR, C_MR, L_CS_d_t, L_CL_d_t):
    spec_MR = C_MR

    if spec_MR['type'] == '設置しない' or spec_MR['type'] == 'その他':
        spec_MR = get_default_cooling_spec()

    E_E_C_d_t_i = calc_E_E_C_hs_d_t_i(1, spec_MR, region, A_A, A_MR, A_OR, L_CS_d_t, L_CL_d_t)

    return E_E_C_d_t_i


# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器のガス消費量
def get_E_G_C_hs_MR_d_t():
    return rac.get_E_G_C_d_t()


# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器の灯油消費量
def get_E_K_C_hs_MR_d_t():
    return rac.get_E_K_C_d_t()


# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器のその他の一次エネルギー消費量
def get_E_M_C_hs_MR_d_t():
    return rac.get_E_M_C_d_t()

$$
\large
E_{E,C,hs,OR,d,t} = \sum_{i=2～5} E_{E,C,hs,d,t,i} 
$$

$$
\large
E_{G,C,hs,OR,d,t} = \sum_{i=2～5} E_{G,C,hs,d,t,i} 
$$

$$
\large
E_{K,C,hs,OR,d,t} = \sum_{i=2～5} E_{K,C,hs,d,t,i} 
$$

$$
\large
E_{M,C,hs,OR,d,t} = \sum_{i=2～5} E_{M,C,hs,d,t,i} 
$$

In [ ]:
# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器の消費電力
def get_E_E_C_hs_OR_d_t(region, A_A, A_MR, A_OR, C_OR, L_CS_d_t, L_CL_d_t):
    # その他の居室がない場合
    if A_OR == 0:
        return np.zeros(24 * 365)

    spec_OR = C_OR

    if spec_OR['type'] == '設置しない' or spec_OR['type'] == 'その他':
        spec_OR = get_default_cooling_spec()

    tmp = []
    for i in [3, 4, 5]:
        E_E_C_d_t_i = calc_E_E_C_hs_d_t_i(i, spec_OR, region, A_A, A_MR, A_OR, L_CS_d_t, L_CL_d_t)
        tmp.append(E_E_C_d_t_i)

    return np.sum(tmp, axis=0)


def calc_E_E_C_hs_d_t_i(i, device, region, A_A, A_MR, A_OR, L_CS_d_t, L_CL_d_t):
    if device['type'] == 'ルームエアコンディショナー':

        # 仕様の取得
        A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)
        q_rtd_C = rac_spec.get_q_rtd_C(A_HCZ_i)
        e_rtd_C = rac_spec.get_e_rtd_C(device['e_class'], q_rtd_C)

        # 電力消費量の計算
        E_E_C_d_t_i = rac.get_E_E_C_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            e_rtd_C=e_rtd_C,
            dualcompressor=device['dualcompressor'],
            L_CS_d_t=L_CS_d_t[i - 1],
            L_CL_d_t=L_CL_d_t[i - 1]
        )
    else:
        raise ValueError(device['type'])

    return E_E_C_d_t_i


# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器のガス消費量
def get_E_G_C_hs_OR_d_t():
    return np.sum([rac.get_E_G_C_d_t() for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器の灯油消費量
def get_E_K_C_hs_OR_d_t():
    return np.sum([rac.get_E_K_C_d_t() for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器のその他の一次エネルギー消費量
def get_E_M_C_hs_OR_d_t():
    return np.sum([rac.get_E_M_C_d_t() for i in range(2, 6)], axis=0)

## 付録 A 設置する暖房設備機器又は放熱器の種類に応じた暖房方式及び運転方法の決定方法並びに評価上想定される暖房設備機器又は放熱器の種類 / Appendix A How to determine the heating method and operation method according to the type of heating equipment or radiator to be installed and the type of heating equipment or radiator assumed for evaluation

### A.1 設置する暖房設備機器又は放熱器の種類に応じた暖房方式及び運転方法の決定方法 / Method of determining heating method and operation method according to the type of heating equipment or radiator to be installed

In [ ]:
def get_heating_mode(region, H_A=None, H_MR=None, H_OR=None):
    if H_A is None:

        if H_MR is None and H_OR is None:
            return None, None

        default_spec = get_default_heating_spec(region)

        if H_MR['type'] in ['設置しない', 'その他']:
            H_MR_type = default_spec[0]['type']
        else:
            H_MR_type = H_MR['type']

        if H_OR is not None:
            if H_OR['type'] in ['設置しない', 'その他']:
                H_OR_type = default_spec[1]['type']
            else:
                H_OR_type = H_OR['type']

        y = get_index_of_table_a_1(H_MR_type)

        def to_roha(s):
            if type(s) is tuple:
                return (to_roha(s[0]), to_roha(s[1]))
            else:
                if s == '連続':
                    return 'ろ'
                elif s == '間歇':
                    return 'は'
                else:
                    raise ValueError(s)

        if H_OR is not None:
            x = get_index_of_table_a_1(H_OR_type)
            return to_roha(table_a_1_a[y][x])
        else:
            tmp = table_a_1_b[y]
            if type(tmp) is tuple:
                if region in [1, 2]:
                    return to_roha(tmp[0]), None
                elif region in [3, 4, 5, 6, 7]:
                    return to_roha(tmp[1]), None
                else:
                    raise ValueError(region)
            else:
                return to_roha(tmp), None

    if H_A['type'] == 'ダクト式セントラル空調機':
        # 住宅全体を連続的に暖房する方式
        return 'い'

    else:
        raise ValueError(H_A['type'])


def get_index_of_table_a_1(type):
    key_table = {
        '電気蓄熱暖房器': 0,
        '温水暖房用パネルラジエーター': 1,
        '温水暖房用床暖房': 2,
        '温水暖房用ファンコンベクター': 3,
        'ルームエアコンディショナー': 4,
        'FF暖房機': 5,
        '電気ヒーター床暖房': 6,
        'ルームエアコンディショナー付温水床暖房機': 7
    }
    return key_table[type]

<div style="text-align: center;font-weight: bold;">表　主たる居室及びその他の居室の運転方法（その他の居室がある場合）<br>（上段：主たる居室の運転方法 下段：その他の居室の運転方法）<br>Table: Operating method of main living room and other living rooms (when there are other living rooms) <br> (upper: operating method of main living room, lower: operating method of other living rooms)</div>

<table>
<tr>
<th colspan="2" rowspan="2"></th>
<th colspan="8">その他の居室に設置する暖冷房設備機器等<br>Heating and cooling equipment installed in other rooms</th>
</tr>
<tr>
<th>電気蓄熱<br>暖房器<br>Electric heat storage heater</th>
<th>パネル<br>ラジエーター<br>Panel radiator</th>
<th>温水床暖房<br>Heated floor heating</th>
<th>ファン<br>コンベクター<br>Fan convector</th>
<th>ルーム<br>エアコン<br>ディショナー<br>Room air conditioner</th>
<th>FF暖房機<br>FF heater</th>
<th>電気<br>ヒーター<br>床暖房<br>Electric heater floor heating</th>
<th>ルーム<br>エアコン<br>ディショナー付<br>温水床暖房<br>Hot-water floor heating with room air conditioner</th>
</tr>

<tr>
<th rowspan="8">主たる居室に<br>設置する<br>暖冷房<br>設備機器等<br>Heating and cooling equipment installed in the main room</th>
<th>電気蓄熱暖房器<br>Electric heat storage heater</th>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
</tr>
<tr>
<th>パネルラジエーター<br>Panel radiator</th>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
<td>連続<br>Continuous<br>間歇<br>Intermittent</td>
</tr>
<tr>
<th>温水床暖房<br>Heated floor heating</th>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>連続<br>Continuous<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
</tr>
<tr>
<th>ファンコンベクター<br>Fan convector</th>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
</tr>
<tr>
<th>ルームエア<br>コンディショナー<br>Room air conditioner</th>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
</tr>
<tr>
<th>FF 暖房機<br>FF heater</th>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
</tr>
<tr>
<th>電気ヒーター床暖房<br>Electric heater floor heating</th>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
</tr>
<tr>
<th>ルーム<br>エアコン<br>ディショナー付<br>温水床暖房<br>Hot-water floor heating with room air conditioner</th>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>連続<br>Continuous</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
<td>間歇<br>Intermittent<br>間歇<br>Intermittent</td>
</tr>
</table>

In [ ]:
# 表 主たる居室及びその他の居室の運転方法(その他の居室がある場合)
table_a_1_a = [
    # 電気蓄熱暖房器
    (
        ('連続', '連続'),
        ('連続', '連続'),
        ('連続', '連続'),
        ('連続', '間歇'),
        ('連続', '間歇'),
        ('連続', '間歇'),
        ('連続', '間歇'),
        ('連続', '間歇'),
    ),
    # パネルラジエータ―
    (
        ('連続', '連続'),
        ('連続', '連続'),
        ('連続', '連続'),
        ('連続', '間歇'),
        ('連続', '間歇'),
        ('連続', '間歇'),
        ('連続', '間歇'),
        ('連続', '間歇'),
    ),
    # 温水床暖房
    (
        ('連続', '連続'),
        ('連続', '連続'),
        ('連続', '連続'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
    ),
    # ファンコンベクター
    (
        ('間歇', '連続'),
        ('間歇', '連続'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
    ),
    # ルームエアコンディショナー
    (
        ('間歇', '連続'),
        ('間歇', '連続'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
    ),
    # FF暖房機
    (
        ('間歇', '連続'),
        ('間歇', '連続'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
    ),
    # 電気ヒーター床暖房
    (
        ('間歇', '連続'),
        ('間歇', '連続'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
    ),
    # ルームエアコンディショナー付温水床暖房
    (
        ('間歇', '連続'),
        ('間歇', '連続'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
        ('間歇', '間歇'),
    )
]


<div style="text-align: center;font-weight: bold;">表　主たる居室の運転方法（その他の居室がない場合）<br>Table: Main room operation method (when there are no other rooms)</div>

<table>
<tr>
<th colspan="3"></th>
<th>運転方法<br>how to drive</th>
</tr>
<tr>
<th rowspan="9">主たる居室に設置する<br>暖冷房設備機器等<br>Heating and cooling equipment<br>installed in the main room</th>
<th colspan="2">電気蓄熱暖房器<br>Electric heat storage heater</th>
<td>連続<br>Continuous<br>Continuous</td>
</tr>
<tr>
<th colspan="2">パネルラジエーター<br>Panel radiator</th>
<td>連続<br>Continuous<br>Continuous</td>
</tr>
<tr>
<th rowspan="2">温水床暖房<br>Heated floor heating</th>
<th>1～2 地域<br>1-2 area</th>
<td>連続<br>Continuous<br>Continuous</td>
</tr>
<tr>
<th>3～7 地域<br>3-7 area</th>
<td>間歇<br>Intermittent</td>
</tr>
<tr>
<th colspan="2">ファンコンベクター<br>Fan convector</th>
<td>間歇<br>Intermittent</td>
</tr>
<tr>
<th colspan="2">ルームエアコンディショナー<br>Room air conditioner</th>
<td>間歇<br>Intermittent</td>
</tr>
<tr>
<th colspan="2">FF 暖房機<br>FF heater</th>
<td>間歇<br>Intermittent</td>
</tr>
<tr>
<th colspan="2">電気ヒーター床暖房<br>Electric heater floor heating</th>
<td>間歇<br>Intermittent</td>
</tr>
<tr>
<th colspan="2">ルームエアコンディショナー付<br>温水床暖房<br>Hot-water floor heating with room air conditioner</th>
<td>間歇<br>Intermittent</td>
</tr>
</table>

In [ ]:
# 表 主たる居室の運転方法(その他の居室がない場合)
table_a_1_b = [
    '連続',
    '連続',
    ('連続', '間歇'),
    '間歇',
    '間歇',
    '間歇',
    '間歇',
    '間歇',
]

### A.2 複数の暖房設備機器等が設置される場合の評価上想定される暖房設備機器等の種類 / Types of heating equipment etc. assumed for evaluation when multiple heating equipment etc. are installed

省略

Omission

### A.3 暖房設備機器等が設置されない場合の評価上想定される暖房設備機器等の種類 / Types of heating equipment and equipment assumed for evaluation when heating equipment and equipment etc. are not installed

主たる居室若しくはその他の居室に暖房設備機器等を設置しない場合又は表に掲げる暖房設備機器等以外の暖房設備機器等を設置する場合は、地域の区分に応じて表に示す暖房設備機器等が設置されるものとして評価する。その際、パネルラジエーターが想定された場合（1 地域及び 2 地域）の配管の断熱措置については「断熱被覆のないもの」とし、温水暖房用熱源機については石油温水暖房機で、定格能力効率を0.830(83.0%)として評価する。また、FF 暖房機が想定された場合（3 地域及び 4 地域）の FF 暖房機の定格能力におけるエネルギー消費効率は0.860(86.0%)とする。ルームエアコンディショナーが想定された場合（5～7地域）のルームエアコンディショナーのエネルギー消費効率の区分は区分（ろ）とする。 

If you do not install heating equipment etc. in the main room or other living rooms, or if you install heating equipment other than the heating equipment etc. listed in the table, then the heating equipment etc. shown in the table is installed according to the division of the area To be evaluated. At that time, if panel radiators are assumed (1 area and 2 areas), “insulation without insulation coating” is assumed for piping insulation measures, and for oil heating units, oil water heaters are used to provide rated capacity efficiency It is evaluated as 0.830 (83.0%). The energy consumption efficiency at the rated capacity of the FF heater is assumed to be 0.860 (86.0%) when the FF heater is assumed (three regions and four regions). When room air conditioners are assumed (5 to 7 areas), division of energy consumption efficiency of room air conditioners is classified as division (RO).

In [ ]:
# 暖房設備機器等が設置されない場合の評価上想定される暖房設備機器等
def get_default_heating_spec(region):
    def to_spec(type):
        if type == '温水暖房用パネルラジエーター':
            return {
                'type': type
            }
        elif type == 'FF暖房機':
            return {
                'type': type,
                'e_rtd': 0.86
            }
        elif type == 'ルームエアコンディショナー':
            return {
                'type': type,
                'e_class': 'ろ',
                'dualcompressor': False
            }
        else:
            raise ValueError(type)

    if region != 8:
        type_list = table_a_5[region - 1]

        if region in [1, 2]:
            hw_spec = {
                'type': '石油従来型温水暖房機',
                'e_rtd': 0.83
            }
            return to_spec(type_list[0]), to_spec(type_list[1]), hw_spec
        else:
            return to_spec(type_list[0]), to_spec(type_list[1]), None
    else:
        return None, None, None

<div style="text-align: center;font-weight: bold;">表　主たる居室若しくはその他の居室に暖房設備機器等を設置しない場合又は表 A.1 に掲げる<br>暖房設備機器等以外の暖房設備機器等を設置する場合の評価において想定する暖房設備機器等<br>Table: Heating equipment and equipment assumed in evaluation when heating equipment and equipment other than heating equipment and equipment listed in Table A.1 are not installed in the main room or other living room</div>

<table>
<thead>
<tr>
	<th rowspan="2">地域の区分<br>Segmentation of area</th>
	<th colspan="2">評価において想定される暖房設備機器等<br>Heating equipment etc. assumed in evaluation</th>
</tr>
<tr>
	<th>主たる居室<br>Main room</th>
	<th>その他の居室<br>Other rooms</th>
</tr>
</thead>
<tbody>
<tr>
	<th>1</th>
	<td>パネルラジエーター<br>Panel radiator</td>
	<td>パネルラジエーター<br>Panel radiator</td>
</tr>
<tr>
	<th>2</th>
	<td>パネルラジエーター<br>Panel radiator</td>
	<td>パネルラジエーター<br>Panel radiator</td>
</tr>
<tr>
	<th>3</th>
	<td>ＦＦ暖房機<br>FF heater</td>
	<td>ＦＦ暖房機<br>FF heater</td>
</tr>
<tr>
	<th>4</th>
	<td>ＦＦ暖房機<br>FF heater</td>
	<td>ＦＦ暖房機<br>FF heater</td>
</tr>
<tr>
	<th>5</th>
	<td>ルームエアコンディショナー<br>Room air conditioner</td>
	<td>ルームエアコンディショナー<br>Room air conditioner</td>
</tr>
<tr>
	<th>6</th>
	<td>ルームエアコンディショナー<br>Room air conditioner</td>
	<td>ルームエアコンディショナー<br>Room air conditioner</td>
</tr>
<tr>
	<th>7</th>
	<td>ルームエアコンディショナー<br>Room air conditioner</td>
	<td>ルームエアコンディショナー<br>Room air conditioner</td>
</tr>
</tbody>
</table>

In [ ]:
# 表 主たる居室若しくはその他の居室に暖房設備機器等を設置しない場合又は表 A.1 に掲げる 暖房設備機器等以外の暖房設備機器等を設置する場合の評価において想定する暖房設備機器等
table_a_5 = [
    ('温水暖房用パネルラジエーター', '温水暖房用パネルラジエーター'),
    ('温水暖房用パネルラジエーター', '温水暖房用パネルラジエーター'),
    ('FF暖房機', 'FF暖房機'),
    ('FF暖房機', 'FF暖房機'),
    ('ルームエアコンディショナー', 'ルームエアコンディショナー'),
    ('ルームエアコンディショナー', 'ルームエアコンディショナー'),
    ('ルームエアコンディショナー', 'ルームエアコンディショナー'),
]

主たる居室若しくはその他の居室又はその両方に、温水床暖房、ファンコンベクター又はパネルラジエーターが設置された場合において、配管を設置しない場合においては、配管の断熱措置を「断熱被覆のないもの」として評価し、温水暖房用熱源機を設置しない場合又はその他の温水暖房用熱源機を設置する場合においては、表 A.6 に示す温水暖房用熱源機が設置されたものとして評価する。その場合の定格効率として、石油従来型温水暖房機の場合は0.830(83.0%)、ガス従来型温水暖房機の場合は0.825(82.5％)で評価する。

If hot water floor heating, fancon vector or panel radiators are installed in the main room and / or other rooms, the piping insulation measures are evaluated as "without thermal insulation coating" in the case where piping is not installed. When the hot water heating heat source machine is not installed or when other hot water heating heat source machines are installed, it is evaluated that the hot water heating heat source machine shown in Table A.6 is installed. As rated efficiency in that case, in the case of petroleum conventional hot water heater, it evaluates by 0.830 (83.0%), and, in the case of gas conventional hot water heater, by 0.825 (82.5%).

In [ ]:
# 温水暖房用熱源機を設置しない場合又はその他の温水暖房機を設置する場合に想定する温水暖房用熱源機の仕様
def get_default_heatsource(region):
    if region == 8:
        return None

    hs_type = table_a_6[region - 1]

    e_rtd_table = {
        '石油従来型温水暖房機': 0.830,
        'ガス従来型温水暖房機': 0.825
    }
    e_rtd = e_rtd_table[hs_type]

    return {
        'type': hs_type,
        'e_rtd_hs': e_rtd,

        # 配管を設置しない場合においては、配管の断熱措置を「断熱被覆のないもの」として評価
        'pipe_insulation': False,
        'underfloor_pipe_insulation': False
    }

<div style="text-align: center;font-weight: bold;">表　温水暖房用熱源機を設置しない又はその他の温水暖房用熱源機を<br>設置する場合の評価において想定する温水暖房用熱源機<br>Table: Hot water heating heat source machine assumed in the evaluation when no hot water heating heat source machine is installed or other hot water heating heat source machine is installed</div>

<table>
<thead>
<tr>
	<th>地域の区分<br>Segmentation of area</th>
	<th>評価において想定される暖房設備機器等<br>Heating equipment etc. assumed in evaluation</th>
</tr>
</thead>
<tbody>
<tr>
	<th>1</th>
	<td>石油従来型温水暖房機<br>Petroleum conventional hot water heater</td>
</tr>
<tr>
	<th>2</th>
	<td>石油従来型温水暖房機<br>Petroleum conventional hot water heater</td>
</tr>
<tr>
	<th>3</th>
	<td>石油従来型温水暖房機<br>Petroleum conventional hot water heater</td>
</tr>
<tr>
	<th>4</th>
	<td>石油従来型温水暖房機<br>Petroleum conventional hot water heater</td>
</tr>
<tr>
	<th>5</th>
	<td>ガス従来型温水暖房機<br>Gas conventional hot water heater</td>
</tr>
<tr>
	<th>6</th>
	<td>ガス従来型温水暖房機<br>Gas conventional hot water heater</td>
</tr>
<tr>
	<th>7</th>
	<td>ガス従来型温水暖房機<br>Gas conventional hot water heater</td>
</tr>
</tbody>
</table>

In [ ]:
# 表 温水暖房用熱源機を設置しない又はその他の温水暖房用熱源機を設置する場合の評価において想定する温水暖房用熱源機
table_a_6 = [
    '石油従来型温水暖房機',
    '石油従来型温水暖房機',
    '石油従来型温水暖房機',
    '石油従来型温水暖房機',
    'ガス従来型温水暖房機',
    'ガス従来型温水暖房機',
    'ガス従来型温水暖房機',
]

## 付録 B 設置する冷房設備機器の種類に応じた冷房方式及び運転方法の決定方法並びに評価上想定される冷房設備機器の種類 / Appendix B How to determine the cooling method and operation method according to the type of cooling equipment to be installed and the type of cooling equipment assumed for evaluation 

「居室のみを冷房する方式」の場合の主たる居室及びその他の居室における運転方法はともに間歇運転とする。  
主たる居室、その他の居室に冷房設備機器を設置しない場合又はルームエアコンディショナー以外の冷房設備機器を設置する場合は、ルームエアコンディショナーが設置されたものとして評価する。その際、ルームエアコンディショナーのエネルギー消費効率の区分は区分（ろ）とする。 

The operation methods in the main room and the other rooms in the case of the "system for cooling only the room" are both intermittent.
If no cooling equipment is installed in the main room or other rooms, or if cooling installations other than the room air conditioner are installed, the room air conditioner is evaluated as installed. At that time, division of energy consumption efficiency of room air conditioner is classified as division (RO).

In [ ]:
def get_default_cooling_spec():
    return {
        'type': 'ルームエアコンディショナー',
        'e_class': 'ろ',
        'dualcompressor': False,
    }

In [ ]:
'''
試算例の作成
'''
if __name__ == '__main__':

    region = 6
    A_A = 120
    A_MR = 30
    A_OR = 50
    A_env = 300
    Q = 0.87 * A_env / A_A
    mu_H = 2.4 / 100 * A_env / A_A
    mu_C = 3.5 / 100 * A_env / A_A
    mode_C = '住戸全体を連続的に冷房する方式'
    # C_A = {'P_rtd_C' : 1500, 'C_pl_C' : 1, 'EquipmentSpec' : '入力しない', 'VAV' : False}
    C_A = {
        'type': 'ダクト式セントラル空調機',
        'duct_insulation': '全てもしくは一部が断熱区画外である',
        'VAV': False,
        'general_ventilation': True,
        'EquipmentSpec': '入力しない'
    }

    C_MR = {'type' : 'ダクト式セントラル空調'}
    C_OR = {'type' : 'ダクト式セントラル空調'}
    L_H_d_t_i = np.array([np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24),
                          np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24),
                          np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24)])
    L_CS_d_t_i = np.array([np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24),
                          np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24),
                          np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24)])
    L_CL_d_t_i = np.array([np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24),
                          np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24),
                          np.ones(365 * 24), np.ones(365 * 24), np.ones(365 * 24)])

    # L_H_d_t = np.sum(L_H_d_t_i, axis=0)
    # L_CS_d_t = np.sum(L_CS_d_t_i, axis=0)
    # L_CL_d_t = np.sum(L_CL_d_t_i, axis=0)

    print('4-1-f')
    '''L_CS_d_t_i,L_CL_d_t_iを指定する必要があるかもしれない。
    この式から動かすのはよくないかもしれない。'''
    test1 = calc_E_E_C_d_t(region, A_A, A_MR, A_OR, A_env, mu_H, mu_C, Q, C_A, C_MR, C_OR, L_H_d_t_i,
                           L_CS_d_t_i, L_CL_d_t_i)

    print('test1の結果は{}'.format(np.sum(test1)))
